In [2]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

matlab_uq8665


In [3]:
# 显示cell运行时长
%load_ext klab-autotime

In [4]:
!pip install mat4py

time: 3.16 s


In [5]:
import mat4py
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import time
import plotly.express as px



Start = time.time()

f_name = '/home/mw/input/matlab_uq8665/res.mat'
mat_data = mat4py.loadmat(f_name)
mat_Data = pd.DataFrame(mat_data)
mat_Data.drop('RESITER_cell', axis=1, inplace=True)
mat_Data

RESLOAD_cell = []


def RESLOAD(index):

    for i in mat_Data['RESLOAD_cell'][index]:
        for s in i[20:]:
            for q in s:
                RESLOAD_cell.append(q)
        #RESLOAD_cell_TOTAL.append(RESLOAD_cell)

    return RESLOAD_cell


for index in range(12):
    RESLOAD(index)

RESU_cell = []


def RESU(index):

    for i in mat_Data['RESU_cell'][index]:
        for s in i[20:]:
            for q in s:
                RESU_cell.append(q)
        #RESLOAD_cell_TOTAL.append(RESLOAD_cell)

    return RESU_cell


for index in range(12):
    RESU(index)

P = []
t = 1
for s in range(12):
    i = 0
    while i < 20:
        P.append(t)
        i += 1
    t += 1

Table = pd.DataFrame({'LOAD': RESLOAD_cell, 'P': P, 'U': RESU_cell}, )
#Table.sort_values(by=['LOAD', 'P'], ascending=True, inplace=True)


#%%

x = Table.P
y = Table.LOAD
z = Table.U

print(len(x))

fig = plt.figure(figsize=(10, 10))
ax = Axes3D(fig)

ax.scatter(x, y, z, s=50)
plt.xlabel('P', fontsize='30', labelpad=30)
plt.ylabel('LOAD', fontsize='30', labelpad=30)
ax.set_zlabel('U', fontsize='30', labelpad=30)
plt.show()

#px.scatter_3d(x=Table.P, y=Table.LOAD, z=Table.U)

240


<Figure size 720x720 with 1 Axes>

time: 1.89 s


In [9]:
import matplotlib.cm as cm
from scipy import interpolate

Func_U = interpolate.interp2d(Table.P, Table.LOAD, Table.U, kind='cubic')


#计算480*480的网格上的插值
Table_test = pd.DataFrame({'LOAD': Table.LOAD, 'P': Table.P}, )
print(Table_test.size)


Table_test_2 = pd.DataFrame({'LOAD': Table.LOAD, 'P': Table.P}, )
Table_New=Table_test.append(Table_test_2, ignore_index=True)
Table_New.sort_values(by=[ 'P'], ascending=True, inplace=True)




New_U = Func_U(Table_New.P, Table_New.LOAD)
New_LOAD = Table_New.LOAD
New_P = Table_New.P

xnew = New_P
ynew = New_LOAD
znew = New_U

fig = plt.figure(figsize=(20, 10))
# ax = Axes3D(fig)

# ax.scatter(x, y, z, s=100)
# plt.xlabel('P', fontsize='30', labelpad=30)
# plt.ylabel('LOAD', fontsize='30', labelpad=30)
# ax.set_zlabel('U', fontsize='30', labelpad=30)
# plt.show()

ax2 = plt.subplot(1, 2, 2, projection='3d')
surf2 = ax2.plot_surface(xnew,
                         ynew,
                         znew,
                         rstride=2,
                         cstride=2,
                         cmap=cm.coolwarm,
                         linewidth=0.5,
                         antialiased=True)
ax2.set_xlabel('xnew')
ax2.set_ylabel('ynew')
ax2.set_zlabel('fnew(x, y)')
#plt.colorbar(surf2, shrink=0.5, aspect=5)  #标注

plt.show()

480


<Figure size 1440x720 with 1 Axes>

time: 5.63 s


# scipy interpolation
Here we are trying to achieve two goals. Firstly, a interpolation function needs to be created. There are many ways you can create the interpolation fuction. One of the easy method is using rbf kernel, the same math behind Gaussian processes, demonstrated in the official tutorial from scipy. [2 D interpolation](https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html)

In [43]:
import numpy as np
from scipy.interpolate import Rbf
import matplotlib.pyplot as plt
from matplotlib import cm


x = Table.P
y = Table.LOAD
z = Table.U
XI, YI = np.meshgrid(np.linspace(0,Table.P.max(),101), np.linspace(0,Table.LOAD.max(),101))

# use RBF epsilon is the magic number 
rbf = Rbf(x, y, z, epsilon=1)
ZI = rbf(XI, YI)

time: 272 ms


In [53]:
print(f'{rbf(x[0],y[0]) = :.5f}')
print(f'{z[0] = :.5f}')

rbf(x[0],y[0]) = 0.00053
z[0] = 0.00053
time: 998 µs


In [54]:
rbf(0.5,0.5)

array(0.00122612)

time: 1.86 ms


Secondly, we need to give a qualitative analysis of our interpolation. Also, there are more than one way of creating the visualization. Since the problem we are looking at is 3 dimenstional( 2 inputs and 1 output). One of the easy visualisation is 2d color contour plot. 

In [41]:
# plot the result
plt.subplot(1, 1, 1)
lims = dict(cmap='RdBu_r')
# lims = dict(cmap='viridis')

plt.pcolormesh(XI,YI,ZI,shading='flat',**lims)
plt.scatter(x, y,100, z, edgecolor='w', lw=0.1, **lims)
plt.title('RBF interpolation - multiquadrics')
plt.xlabel('p')
plt.ylabel('load')
plt.colorbar()

<Figure size 432x288 with 2 Axes>

time: 169 ms


## 2. Multivariate data interpolation (griddata)
Multivariate data interpolation (griddata)
Suppose you have multidimensional data, for instance, for an underlying function f(x, y) you only know the values at points (x[i], y[i]) that do not form a regular grid.

Suppose we want to interpolate the 2-D function

In [57]:
grid_x, grid_y = np.mgrid[0:Table.P.max():100j, 0:Table.LOAD.max():100j]
points = Table[['P','LOAD']].values
values = Table['U'].values

time: 2.64 ms


In [58]:
from scipy.interpolate import griddata
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')
grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic')

time: 15.2 ms


In [59]:
import matplotlib.pyplot as plt
plt.subplot(221)
# plt.imshow(func(grid_x, grid_y).T, extent=(0,1,0,1), origin='lower')
plt.plot(points[:,0], points[:,1], 'k.', ms=1)
plt.title('Original')
plt.subplot(222)
plt.imshow(grid_z0.T, extent=(0,1,0,1), origin='lower')
plt.title('Nearest')
plt.subplot(223)
plt.imshow(grid_z1.T, extent=(0,1,0,1), origin='lower')
plt.title('Linear')
plt.subplot(224)
plt.imshow(grid_z2.T, extent=(0,1,0,1), origin='lower')
plt.title('Cubic')
plt.gcf().set_size_inches(6, 6)
plt.show()

<Figure size 432x432 with 4 Axes>

time: 377 ms


In [64]:
x,y=np.mgrid[1:1:1j,1:1:1j]
print(f'{x,y=}')

x,y=(array([[1.]]), array([[1.]]))
time: 1.1 ms


In [65]:
griddata(points, values, (x, y), method='nearest')

array([[0.0013332]])

time: 2.12 ms


In [72]:
def grid_linear(x,y):
    return griddata(points, values, (x,y), method='linear')

grid_linear(1,1)

array(0.00140411)

time: 5.45 ms


## [interp2d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html?highlight=interp2d) is difficult to use for it requires the input data is griddata. 

[知乎scipy插值](https://zhuanlan.zhihu.com/p/136700122)

In [50]:
from scipy import interpolate
x = np.arange(-5.01, 5.01, 0.25)
y = np.arange(-5.01, 5.01, 0.25)
xx, yy = np.meshgrid(x, y)
z = np.sin(xx**2+yy**2)
f = interpolate.interp2d(x, y, z, kind='cubic')

time: 2.38 ms


In [53]:
x.shape

(41,)

time: 1.45 ms


In [84]:
import numpy as np 
def func(x,y): 
    return (x+y)*np.exp(-5*(x**2+y**2)) 
x,y=np.mgrid[-1:1:8j,-1:1:8j] 
z=func(x,y)

time: 707 µs


In [98]:
from scipy import interpolate
func=interpolate.interp2d(x,y,z,kind='cubic')

xnew=np.linspace(-1,1,100)
ynew=np.linspace(-1,1,100)
znew=func(xnew,ynew)#xnew, ynew是一维的，输出znew是二维的
xnew,ynew=np.mgrid[-1:1:100j,-1:1:100j]#统一变成二维，便于下一步画图

time: 2.36 ms


In [120]:
from scipy import interpolate
x = Table.P
y = Table.LOAD
z = Table.U
func=interpolate.interp2d(x,y,z,kind='cubic')
xnew=np.linspace(0,Table.P.max(),100)
ynew=np.linspace(0,Table.LOAD.max(),100)
znew=func(xnew,ynew)#xnew, ynew是一维的，输出znew是二维的
xnew,ynew=np.mgrid[0:Table.P.max():100j,0:Table.LOAD.max():100j]#统一变成二维，便于下一步画图


time: 14.1 ms


In [121]:
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt
ax=plt.subplot(111,projection='3d')

ax.plot_surface(xnew,ynew,znew)
ax.scatter(x,y,z,c='r',marker='^')
plt.show()

<Figure size 432x288 with 1 Axes>

time: 606 ms


## RBF 3d plot

In [122]:
xnew=np.linspace(0,Table.P.max(),100)
ynew=np.linspace(0,Table.LOAD.max(),100)
xnew,ynew=np.mgrid[0:Table.P.max():100j,0:Table.LOAD.max():100j]
znew=rbf(xnew,ynew)


time: 27.7 ms


In [123]:
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt
ax=plt.subplot(111,projection='3d')

ax.plot_surface(xnew,ynew,znew)
ax.scatter(x,y,z,c='r',marker='^')
plt.show()

<Figure size 432x288 with 1 Axes>

time: 560 ms


## griddata 3d plot

In [125]:
xnew=np.linspace(0,Table.P.max(),100)
ynew=np.linspace(0,Table.LOAD.max(),100)
xnew,ynew=np.mgrid[0:Table.P.max():100j,0:Table.LOAD.max():100j]
znew=griddata(points, values, (xnew, ynew), method='linear')

time: 5.79 ms


In [126]:
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt
ax=plt.subplot(111,projection='3d')

ax.plot_surface(xnew,ynew,znew)
ax.scatter(x,y,z,c='r',marker='^')
plt.show()

<ipython-input-126-132b2fef4e77>:5: UserWarning: Z contains NaN values. This may result in rendering artifacts.
  ax.plot_surface(xnew,ynew,znew)


<Figure size 432x288 with 1 Axes>

time: 556 ms


In [69]:
griddata(points, values, (12, 4), method='nearest')

0.006059846334374701

time: 3.17 ms


## scipy 2d interpolation example
[official tutorial](https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html)

In [6]:
import numpy as np
from scipy.interpolate import Rbf
import matplotlib.pyplot as plt
from matplotlib import cm


time: 52.6 ms


In [7]:
# 2-d tests - setup scattered data
rng = np.random.default_rng()
x = rng.random(100)*4.0-2.0
y = rng.random(100)*4.0-2.0
z = x*np.exp(-x**2-y**2)
edges = np.linspace(-2.0, 2.0, 101)
centers = edges[:-1] + np.diff(edges[:2])[0] / 2.
XI, YI = np.meshgrid(centers, centers)

time: 1.58 ms


In [21]:
x

array([ 1.4711492 , -0.40027586,  0.25900292, -0.35601501,  0.62255683,
       -0.53849351, -0.57733366,  0.88570319,  0.86129682, -0.63094793,
       -1.29997762,  0.67876811,  1.47786219,  0.0591824 , -0.46578755,
       -1.63795112, -1.80781918, -1.83736292, -0.44713578,  0.26213777,
        1.14056582,  0.06928248, -0.17086045,  0.56721032, -0.41404083,
       -1.03205055, -1.67589796, -0.0116725 , -1.58793022,  0.556131  ,
       -0.04592974, -0.76601582, -1.30467091,  0.6661111 ,  1.67717021,
        1.57089474, -0.20948611,  1.2916283 , -1.01542762, -1.95889184,
       -0.08330491,  1.87618883,  1.24620922,  0.71277256, -1.05320447,
        0.61339001,  1.67582208,  1.65751323, -0.45934434, -0.76207707,
       -0.92572467, -1.90868784, -0.98323551, -1.01301538, -0.21361979,
        0.90086589,  1.31541121, -0.52854195, -1.00160283, -1.98059054,
       -0.42857533, -1.91522769,  1.11705747, -1.07049799, -0.50525995,
       -0.44550364, -0.84650385, -0.1430718 ,  0.13300103,  1.42

time: 2.37 ms


In [20]:
Table['P'].values

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12,
       12, 12, 12, 12, 12

time: 1.85 ms


In [25]:
centers.shape

(100,)

time: 1.29 ms


In [28]:
XI.shape

(100, 100)

time: 1.38 ms


In [8]:
# use RBF
rbf = Rbf(x, y, z, epsilon=2)
ZI = rbf(XI, YI)

time: 244 ms


In [33]:
ZI

array([[0.00150446, 0.00129248, 0.00109481, ..., 0.00021254, 0.00023458,
        0.0002721 ],
       [0.00154115, 0.00132973, 0.00113288, ..., 0.00026013, 0.00028134,
        0.00031779],
       [0.00157946, 0.00136866, 0.00117268, ..., 0.0003093 , 0.0003298 ,
        0.00036525],
       ...,
       [0.01791394, 0.01788754, 0.0178669 , ..., 0.00939843, 0.00936176,
        0.0093542 ],
       [0.01827806, 0.01826724, 0.01826403, ..., 0.00975118, 0.00970995,
        0.00969638],
       [0.01863928, 0.01864363, 0.01865738, ..., 0.01011351, 0.01006782,
        0.01004816]])

time: 1.98 ms


In [9]:
# plot the result
plt.subplot(1, 1, 1)
X_edges, Y_edges = np.meshgrid(edges, edges)
lims = dict(cmap='RdBu_r', vmin=-0.4, vmax=0.4)
plt.pcolormesh(X_edges, Y_edges, ZI, shading='flat', **lims)
plt.scatter(x, y, 100, z, edgecolor='w', lw=0.1, **lims)
plt.title('RBF interpolation - multiquadrics')
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.colorbar()

<Figure size 432x288 with 2 Axes>

time: 185 ms
